# INDEED SCRAPING 

In [1]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time

#specify driver path
DRIVER_PATH = r'C:/Users/Jeremy/Desktop/Zurich Project/chromedriver.exe'
driver = webdriver.Chrome(executable_path = DRIVER_PATH)

#Navigate the page
driver.get('https://id.indeed.com')

#initial search
search = driver.find_element_by_id("text-input-where") #Fill the "Where" search column
search.send_keys("Jakarta")
find_jobs = driver.find_element_by_xpath('//form/div[3]/button[@type = "submit"]')
find_jobs.click()

##If Caprcha appears, activate this to give you 180 seconds to solve the captcha manually
#time.sleep(180)

#access advanced search bar
advance_search = driver.find_element_by_xpath("//a[@class = 'sl']")
advance_search.click()

##Fill the advanced search

#Search Data Analyst
column_all = driver.find_element_by_xpath("//fieldset/div[1]/div[2]/input[@id='as_and']")
column_all.send_keys("Data Analyst")

#Not include these words
columns_none = driver.find_element_by_xpath("//fieldset/div[4]/div[2]/input[@id='as_not']")
columns_none.send_keys("Senior Lead")

#Type of Jobs
select_jobs_type = driver.find_element_by_xpath("//fieldset/div[7]/div[2]/select/option[@value='all']")
select_jobs_type.click()

#Display 30 results per page
display_result = driver.find_element_by_xpath("//fieldset[2]/div[3]/div[1]/div[2]/select/option[@value='30']")
display_result.click()

#Find Jobs button
find_button = driver.find_element_by_xpath("//button[@id = 'fj']")
find_button.click()

time.sleep(5) #give time for page to load and notif to pop up
## close pop up
# close_pop_up = driver.find_element_by_xpath("//div[@id='popover-foreground']/div[1]/button[@class='popover-x-button-close icl-CloseButton']")
close_pop_up = driver.find_element_by_id('popover-x')
close_pop_up.click()

# #Next Page Button test
# next_page_btn = driver.find_element_by_xpath('.//a[@aria-label="Next"]')
# next_page_btn.click()

### These codes below not belong to mine
### @author: bonniema
### You can read the description here (https://towardsdatascience.com/web-scraping-job-postings-from-indeed-com-using-selenium-5ae58d155daf)

#This code helps you get relevant information from each job card, and then iterate through all job cards on one page.
#Then go to the next page
#"""

#let the driver wait 3 seconds to locate the element before exiting out
driver.implicitly_wait(3) 

titles=[]
companies=[]
locations=[]
links =[]
reviews=[]
salaries = []
descriptions=[]


for i in range(0,20):
    
    job_card = driver.find_elements_by_xpath('//div[contains(@class,"clickcard")]') #div that contains word "clickcard" in its class 
    
    for job in job_card:
       
    #.  not all companies have review
        try:
            review = job.find_element_by_xpath('.//span[@class="ratingsContent"]').text #extract the text
        except:
            review = "None"
        reviews.append(review)
   #.   not all positions have salary
        try:
            salary = job.find_element_by_xpath('.//span[@class="salaryText"]').text
        except:
            salary = "None"
    #.  tells only to look at the element       
        salaries.append(salary)
        
        try:
            location = job.find_element_by_xpath('.//span[contains(@class,"location")]').text
        except:
            location = "None"
    #.  tells only to look at the element       
        locations.append(location)
        
        try:
            title  = job.find_element_by_xpath('.//h2[@class="title"]//a').text
        except:
            title = job.find_element_by_xpath('.//h2[@class="title"]//a').get_attribute(name="title")
        titles.append(title)
        links.append(job.find_element_by_xpath('.//h2[@class="title"]//a').get_attribute(name="href"))
        companies.append(job.find_element_by_xpath('.//span[@class="company"]').text)
        
    try:
        next_page_btn = driver.find_element_by_xpath('.//a[@aria-label="Next"]')
        next_page_btn.click()
    except:
        print("The Pages is ended here")
        break
    
    print("Page: {}".format(str(i+2)))
        

# #Get the description

for link in links:
    driver.get(link)
    try:
        jd = driver.find_element_by_xpath('//div[@id="jobDescriptionText"]').text
        descriptions.append(jd)
    except:
        descriptions.append("No Info")
    
#     print("Link: {}".format(count()))
    
#Bring them all in one dataframe  
df_da=pd.DataFrame()
df_da['Title']=titles
df_da['Company']=companies
df_da['Location']=locations
df_da['Link']=links
df_da['Review']=reviews
df_da['Salary']=salaries
df_da['Description']=descriptions

# #    #except:
# #         #next_page = driver.find_element_by_xpath('//a[.//span[contains(text(),"Next")]]')
# #         #next_page.click()

Page: 2
Page: 3
Page: 4
Page: 5
The Pages is ended here


In [2]:
df_da

,Title,Company,Location,Link,Review,Salary,Description
0,DATA ANALYST,PT. Sharing Vision Indonesia,Jakarta,https://id.indeed.com/rc/clk?jk=cb2ae21ec61193...,None,None,Berpengalaman dalam pekerjaan terkait big data...
1,Data & System Analyst Staff,Sicepat Ekspres Indonesia PT,Jakarta,https://id.indeed.com/rc/clk?jk=88b41913b425d1...,None,None,Job Descriptions:\nPrepare high level report i...
2,Data & Insight Analyst,Ayoconnect,Jakarta,https://id.indeed.com/rc/clk?jk=bbeeb0379b8bdb...,None,None,Work Timezone:\nData & Insight Analyst\nRespon...
3,Data Technology and Analyst,Ralali.com,Jakarta,https://id.indeed.com/company/ralali.com/jobs/...,None,Rp. 7.000.000 - Rp. 13.000.000 per bulan,"As a Data Technology and Analyst, you will be ..."
4,Data Analyst - Oval,Kompas Gramedia,Jakarta,https://id.indeed.com/rc/clk?jk=4d3950d7208e8f...,4.4,None,Data Analyst will work as a team with Data Sci...
...,...,...,...,...,...,...,...
145,Digital Analyst,Weekend Inc,Jakarta,https://id.indeed.com/rc/clk?jk=d87ef6eba6ed2b...,None,None,No Info
146,"Researcher, Financial Services Platform",GO-JEK,Jakarta,https://id.indeed.com/rc/clk?jk=dabc74f408da51...,3.9,None,No Info
147,Business Intelligence Jr. Manager,Fore Coffee,Jakarta,https://id.indeed.com/rc/clk?jk=7235b552a8b720...,3.5,None,No Info
148,BI Analyst Payment,Reeracoen Indonesia,Jakarta,https://id.indeed.com/rc/clk?jk=05176f4b79be0d...,4.2,None,No Info
